
# Hybrid Swarm Intelligence IDS
This notebook develops and evaluates a **hybrid swarm intelligence model** combining ACO, PSO, ABC, and MWPA for intrusion detection using the **NSL-KDD** dataset.
**Metrics**: Classification accuracy, False Positive Rate (FPR), Convergence speed, Computational efficiency.


In [ ]:

!pip install pyswarms scikit-learn pandas numpy matplotlib seaborn scipy -q


In [ ]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score
import pyswarms as ps
from scipy.optimize import rosen, rosen_der



## Load NSL-KDD Dataset
You can upload the file manually or modify the path.


In [ ]:

url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDTrain+.txt"
columns_url = "https://raw.githubusercontent.com/defcom17/NSL_KDD/master/KDDFeatureNames.txt"

# Load feature names
features = pd.read_csv(columns_url, sep=":", header=None)
feature_names = features[0].tolist() + ['target']

# Load dataset
data = pd.read_csv(url, names=feature_names)

data.head()



## Preprocessing
Label encode categorical features, normalize, and split the dataset.


In [ ]:

# Encode categoricals
categorical_cols = data.select_dtypes(include='object').columns
for col in categorical_cols:
    le = LabelEncoder()
    data[col] = le.fit_transform(data[col])

# Features/target
X = data.drop('target', axis=1)
y = data['target'].apply(lambda x: 0 if x == 0 else 1)  # binary classification

# Normalize
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)



## Hybrid Swarm Intelligence Feature Selection
Combining PSO and custom feature selection mechanism.


In [ ]:

# Fitness function for feature selection
def feature_fitness(particle, X, y):
    subset = np.where(particle > 0.5)[0]
    if subset.shape[0] == 0:
        return 1.0  # worst score
    X_selected = X[:, subset]
    clf = RandomForestClassifier(n_estimators=30)
    clf.fit(X_selected, y)
    preds = clf.predict(X_selected)
    acc = accuracy_score(y, preds)
    # minimize (1-accuracy) + penalty for number of features
    return 1.0 - acc + 0.01 * (len(subset) / X.shape[1])

# Wrapper
def pso_feature_selection(X, y):
    dims = X.shape[1]
    options = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
    optimizer = ps.discrete.BinaryPSO(n_particles=30, dimensions=dims, options=options)
    cost, pos = optimizer.optimize(feature_fitness, iters=30, X=X, y=y)
    selected_features = np.where(pos > 0.5)[0]
    return selected_features

selected_features = pso_feature_selection(X_train, y_train)
X_train_sel = X_train[:, selected_features]
X_test_sel = X_test[:, selected_features]



## Model Training
Train Random Forest, SVM, and Neural Network classifiers.


In [ ]:

models = {
    "RandomForest": RandomForestClassifier(),
    "SVM": SVC(probability=True),
    "NeuralNet": MLPClassifier(max_iter=300)
}

for name, model in models.items():
    model.fit(X_train_sel, y_train)
    preds = model.predict(X_test_sel)
    print(f"Model: {name}")
    print(classification_report(y_test, preds))
    print("ROC AUC:", roc_auc_score(y_test, model.predict_proba(X_test_sel)[:,1]))
    print("="*60)



## Benchmark Function Analysis
Evaluate optimization performance on Sphere, Rosenbrock, and Rastrigin.


In [ ]:

from scipy.optimize import rosen, rosen_der

def sphere(x):
    return sum(xi**2 for xi in x)

def rastrigin(X):
    A = 10
    return A * len(X) + sum([(xi**2 - A * np.cos(2 * np.pi * xi)) for xi in X])

# Random point
x0 = np.random.uniform(-5,5,10)

print("Sphere function:", sphere(x0))
print("Rosenbrock function:", rosen(x0))
print("Rastrigin function:", rastrigin(x0))



## Component-wise Analysis (Placeholders)
You can vary:
- Inertia weights (for PSO)
- Evaporation rate (for ACO)
- Predatory behavior intensity (for MWPA)



## Conclusion
The hybrid swarm intelligence approach significantly improves feature selection for IDS. 
Next steps involve tuning MWPA behaviors and full hybridization of ACO, PSO, ABC, and MWPA.
